# Dédupliquer à partir d'un excel

A partir du fichier excel de cluster dédupliqué validé par Christian, nous créons un acteur chapeau et nous ratachons les acteurs du cluster à ce chapeau

## Déclaration des librairies et des variables

In [34]:
import requests
import uuid

import pandas as pd
from shapely import wkb
from shapely.geometry import Point
from sqlalchemy import create_engine


DATABASE_URL = ''
INTERFACE_HOST = "http://localhost:8000"
# Chemin vers le fichier Excel
FILE_PATH = '../duplicated_commerce_actors_threhsold_0_8_11_09_2024.xlsx'


## Connection à la base de données

In [35]:
# Create the engine
engine = create_engine(DATABASE_URL)

## Récupération des id des sources ALIAPUR et COREPILE

Car on sait par expérience que ces 2 Eco-organismes ont des données plus propre, on préfèrera donc utiliser leur données plutôt que celle des autres Eco-organisme en cas de choix à faire, champ par champ pour créer l'acteur chapeau


In [36]:
# récupération des id des sources aliapur et corepile
query = "SELECT id, code FROM qfdmo_source WHERE code IN ('ALIAPUR','COREPILE');"
sources = pd.read_sql_query(query, engine)
trusted_sources = ['ALIAPUR','COREPILE']
trusted_source_ids = sources[sources['code'].isin(trusted_sources)]['id'].tolist()

## Gestion du fichier de déduplication

On récupère le fichier à dédupliquer localement et on le transforme en dataframe pandas

In [37]:


# Lire le fichier Excel et le convertir en DataFrame
df = pd.read_excel(FILE_PATH)

columns = df.columns

grouped_df = df.groupby('cluster_id').apply(lambda x: x.to_dict("records") if not x.empty else [])

## Compilation des acteurs chapeau

- Regroupement des acteurs par cluster
- Choix de la valeur à appliquer pour chacun des champs

In [38]:
# Ordonner chaque groupe par ordre de confiance selon la liste trusted_sources : 
# D'abord la source ALIAPUR, puis COREPILE, puis les autres sources
grouped_df = grouped_df.apply( 
    lambda x: (
        sorted(x, key=lambda y: 
               trusted_source_ids.index(y['source_id']) 
               if y['source_id'] in trusted_source_ids 
               else len(trusted_sources)
        )
    )
)

# print(grouped_df.head())
acteurs_chapeau_par_cluster = {}
for cluster_id, group in grouped_df.items():
    # Pour chacune des colonnes columns
    # On prend dans l'ordre la valeur de la première ligne (group) si elle est non nulle
    # Puis la valeur de la suivante si elle est non nulle
    # etc
    combined_row = {}
    for column in columns:
        for record in group:
            if pd.notnull(record[column]) and record[column]:
                combined_row[column] = record[column]
                break
    combined_row['identifiant_unique'] = str(uuid.uuid4())
    acteurs_chapeau_par_cluster[cluster_id] = combined_row

#acteurs_chapeau_par_cluster = pd.DataFrame(acteurs_chapeau_par_cluster)
acteurs_chapeau = acteurs_chapeau_par_cluster.values()
acteurs_chapeau_df = pd.DataFrame(acteurs_chapeau)

## Création des acteurs chapeau en base de données

In [54]:
# Fonction qui convertit le champ `location_latlon` en point dans postgis
def convert_to_point(latlon_str):
    latlon_str = latlon_str.strip('()')
    latitude, longitude = map(float, latlon_str.split(','))
    return wkb.dumps(Point(longitude, latitude)).hex()

# Fonction qui normalise le siret
def compute_siret(siret):
    if pd.notna(siret):
        siret = str(int(siret))
        if siret and len(siret) == 14:
            return siret
    return None

# Fonction qui normalise le siret
def compute_codepostal(code_postal):
    if pd.notna(code_postal):
        if isinstance(code_postal, float):
            code_postal = str(int(code_postal))
        if code_postal and len(code_postal) == 5:
            return code_postal
        if code_postal and len(code_postal) == 4:
            return '0' + code_postal
    return None

# Application de la transformation de calcue du champ `location`
acteurs_chapeau_df['location'] = acteurs_chapeau_df['location_latlon'].apply(
    convert_to_point
)

# Suppression des colonnes inutiles
cleaned_acteurs_chapeau_df = acteurs_chapeau_df.drop(
    columns=['cluster_id', 'Review', 'location_latlon', 'siren', 'siret'],
    errors='ignore'
)

cleaned_acteurs_chapeau_df["siret"] = acteurs_chapeau_df["siret"].apply(compute_siret)
cleaned_acteurs_chapeau_df["code_postal"] = acteurs_chapeau_df["code_postal"].apply(
    compute_codepostal
)

# Write to the database
cleaned_acteurs_chapeau_df.to_sql(
    'qfdmo_revisionacteur',
    engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=1000,
)

281

## Créaction et mise à jour des RevisionActeurs

In [51]:
for cluster_id, group in grouped_df.items():
    for record in group:
        # Creation du revision acteur pour chaque acteur à dédupliquer
        response = requests.get(
            f"{INTERFACE_HOST}/qfdmo/getorcreate_revisionacteur/"
            f"{record['identifiant_unique']}"
        )
        # Mise à jour du parent_id avec l'identifiant_unique de l'acteur chapeau
        parent_id = acteurs_chapeau_par_cluster[cluster_id]['identifiant_unique']
        query = f"""
            UPDATE qfdmo_revisionacteur
            SET parent_id = '{parent_id}'
            WHERE identifiant_unique = '{record['identifiant_unique']}';
        """
        engine.execute(query)


# Outils pour tester

Supprimer tous les acteurs chapeau qui viennent d'être créés.

⚠️ Ne pas utiliser hors de l'environnement de développement

In [53]:
# Tooling temporaire


# supprimer les acteurs chapeau identifié pas cleaned_acteurs_chapeau_df['identifiant_unique']
identifiant_uniques = cleaned_acteurs_chapeau_df['identifiant_unique'].tolist()
query = f"UPDATE qfdmo_revisionacteur SET parent_id = NULL WHERE parent_id IN ({', '.join(f"'{id}'" for id in identifiant_uniques)});"
engine.execute(query)
query = f"DELETE FROM qfdmo_revisionacteur WHERE identifiant_unique IN ({', '.join(f"'{id}'" for id in identifiant_uniques)});"
engine.execute(query)


## TODO

- [ ] Récupérer les données des acteurs + revisionacteurs à partir de la base de données 
- [ ] Pensez à regarder si un des acteurs à dédupliquer a déjà un acteur chapeau 